In [20]:
from tensorflow.keras.datasets import mnist
import torch.nn as nn
import torch.optim as optim
import torch

((train_images, train_labels),(test_images,test_labels)) = mnist.load_data()

train_vectors = torch.from_numpy(train_labels[:300]).long()

test_vectors = torch.from_numpy(test_labels[:1]).long()

train_images = torch.from_numpy(train_images[:300]).float()
test_images = torch.from_numpy(test_images[:1]).float()

In [21]:
class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.cnn = nn.Sequential(
        nn.Conv2d(1, 8, 2),
        nn.MaxPool2d(4, 2),
        nn.Conv2d(8, 16, 2),
        nn.MaxPool2d(4, 2),
        nn.Flatten(),
        nn.Linear(256,32),
        nn.ReLU(),
        nn.Linear(32, 10),
        )

    def forward(self,x):
        x = x.reshape(-1,1,28,28)
        return self.cnn(x)


In [22]:
cnn = Model()

loss_func =  nn.CrossEntropyLoss()

optimizer = optim.Adam(cnn.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

In [23]:
for epoch in range(100):
    total_loss = 0
    test_loss = 0
    for i in range(int(len(train_images)/100)):
        optimizer.zero_grad()
        image = train_images[100*i:100*i+100]
        label = train_vectors[100*i:100*i+100]
        output = cnn(image)
        loss = loss_func(output, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print("At epoch", epoch)
    print('loss is ', total_loss)
    print('test loss is ', test_loss)
    scheduler.step()


At epoch 0
loss is  52.593180656433105
test loss is  0
At epoch 1
loss is  19.30389928817749
test loss is  0
At epoch 2
loss is  8.500977993011475
test loss is  0
At epoch 3
loss is  6.530947685241699
test loss is  0
At epoch 4
loss is  5.760303854942322
test loss is  0
At epoch 5
loss is  5.499913334846497
test loss is  0
At epoch 6
loss is  5.37736451625824
test loss is  0
At epoch 7
loss is  5.223302841186523
test loss is  0
At epoch 8
loss is  4.972192406654358
test loss is  0
At epoch 9
loss is  4.762992858886719
test loss is  0
At epoch 10
loss is  4.559014916419983
test loss is  0
At epoch 11
loss is  4.360908389091492
test loss is  0
At epoch 12
loss is  4.217483162879944
test loss is  0
At epoch 13
loss is  4.093434929847717
test loss is  0
At epoch 14
loss is  3.940290927886963
test loss is  0
At epoch 15
loss is  3.770066022872925
test loss is  0
At epoch 16
loss is  3.6198474168777466
test loss is  0
At epoch 17
loss is  3.4898346662521362
test loss is  0
At epoch 18
loss i

In [37]:
import numpy as np
import base64
from PIL import Image
from IPython.display import HTML, display

# ---- Helper: normalize, convert to PIL, scale up ----
def to_pil(img):
    img_uint8 = (255 * (img - img.min()) / (img.max() - img.min())).astype(np.uint8)
    pil_img = Image.fromarray(img_uint8)
    return pil_img

# ---- Helper: display PIL images side by side ----
def show_side_by_side(imgs, size=80, per_row=8):
    html = ""
    for i in range(0, len(imgs), per_row):
        row_imgs = ''.join(
            f'<img src="data:image/png;base64,{base64.b64encode(im._repr_png_()).decode("ascii")}" '
            f'style="margin:2px; width:{size}px;">'
            for im in imgs[i:i+per_row]
        )
        html += f"<div>{row_imgs}</div>"
    display(HTML(html))

# ---- Example: extract 32 feature maps from CNN ----
images = []
conv = cnn.cnn[0]
feature_maps = cnn.cnn[1](conv(train_images[1].reshape(-1, 1, 28, 28)))
conv2 = cnn.cnn[2]
feature_maps2 = cnn.cnn[3](conv2(feature_maps))

for i in range(8):
    fmap = feature_maps[0][i].detach().numpy()
    images.append(to_pil(fmap))

show_side_by_side(images, size=48, per_row=8)

for i in range(16):
    fmap = feature_maps2[0][i].detach().numpy()
    images.append(to_pil(fmap))
show_side_by_side(images, size=16, per_row=8)
# ---- Display all maps in a grid (8 per row) ----
